# Inheco SCILA

| Summary | Image |
|------------|--------|
| <ul style="font-size:15px; line-height:1.6; margin-top:0;"> <li>Automated CO&#8322;-controlled incubator with 4 independently accessible drawers for SBS-format plates.</li> <li><a href="https://www.inheco.com/scila.html" target="_blank"><b>OEM Link</b></a></li> <li><b>Communication Protocol / Hardware:</b> SiLA 2 (SOAP/HTTP) / Ethernet</li> <li><b>Communication Level:</b> SiLA 2 interface (documentation shared by OEM)</li> <li><b>4 independent drawers</b> for SBS-format plates</li> <li>Temperature control (single zone, all drawers)</li> <li>CO&#8322; and H&#8322;O valve monitoring</li> <li>Humidification reservoir level monitoring</li> <li>Only one drawer can be open at a time</li> </ul> | <div style="width:320px; text-align:center;"> ![scila](img/inheco_scila.png) <br><i>Figure: Inheco SCILA</i> </div> |

---
## Setup (Physical)

The SCILA communicates over Ethernet using the SiLA 2 protocol. To connect, you need:
1. The IP address of the SCILA on your network.
2. (Optional) The IP address of your client machine — auto-detected if omitted.

The backend starts a local HTTP server to receive asynchronous responses from the SCILA.

---
## Setup (Programmatic)

In [1]:
import asyncio

In [2]:
from pylabrobot.storage.inheco.scila import SCILABackend

scila = SCILABackend(scila_ip="169.254.1.117")
await scila.setup()

## Usage

---
### Status Requests

Device status (`"standBy"`, `"inError"`, `"startup"`, ...):

In [3]:
await scila.request_status()

'idle'

Water level in the built-in humidification reservoir (e.g. `"High"`, `"Low"`). The SCILA uses this reservoir to maintain humidity inside the drawers:

In [4]:
await scila.request_liquid_level()

'Empty'

Drawer status for all 4 drawers:

In [5]:
await scila.request_drawer_status(1)

'Closed'

In [6]:
await scila.request_drawer_statuses()

{1: 'Closed', 2: 'Closed', 3: 'Closed', 4: 'Closed'}

CO&#8322; and H&#8322;O valve status:

In [7]:
await scila.request_valve_status()

{'H2O': 'Opened', 'CO2 Normal': 'Opened', 'CO2 Boost': 'Closed'}

CO&#8322; flow status:

In [8]:
await scila.request_co2_flow_status()

'NOK'

Status of a single drawer:

In [9]:
await scila.request_drawer_status(3)

'Closed'

---
### Drawer Control

Only one drawer can be open at a time. Opening a second drawer while one is already open will raise an error.

In [10]:
await scila.open(2)

In [11]:
await scila.close(2)

---
### Temperature Control

The SCILA has a single temperature zone shared across all 4 drawers.

Current temperature in °C:

In [12]:
await scila.measure_temperature()

23.65

In [13]:
await scila.start_temperature_control(37.0)

Check the target temperature and current temperature after starting:

In [14]:
await scila.request_target_temperature()

37.0

In [15]:
await asyncio.sleep(4)

await scila.measure_temperature()

24.53

In [16]:
await scila.is_temperature_control_enabled()

True

Stop temperature control and verify it is disabled:

In [17]:
await scila.stop_temperature_control()

In [18]:
await scila.is_temperature_control_enabled()

False

---
## Closing Connection

Close the SiLA 2 HTTP server and disconnect from the SCILA.

In [19]:
await scila.stop()